### Импорты

In [2]:
import sqlite3
import json
import natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [47]:
from pymystem3 import Mystem

In [3]:
import wget

In [4]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatization(token):
    p = morph.parse(token)[0]
    return p.normal_form

In [5]:
import gensim
import gensim.corpora as corpora
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences

In [6]:
from tqdm.auto import tqdm

In [29]:
from threading import Thread

Скачиваем модель русских новостей

In [41]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/20/186.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

100% [......................................................................] 640473374 / 640473374

Маппинг pos-тегов

In [39]:
import requests
import re

url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mapping = {}
r = requests.get(url, stream=True)
for pair in r.text.split('\n'):
    pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
    if len(pair) > 1:
        mapping[pair[0]] = pair[1]

Функция, которая размечает слова для получения синонимов и конвертирует pos-теги

In [74]:
def tag_mystem(text):  
    m = Mystem()
    w = m.analyze(text)[0]
    print(w)
    tagged = []
    try:
        if len(w["analysis"]) == 0:
            tagged.append(text + '_X')
        else:
            lemma = w["analysis"][0]["lex"]
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0]
            if pos in mapping:
                tagged.append(lemma + '_' + mapping[pos]) # здесь мы конвертируем тэги
            else:
                tagged.append(lemma + '_X') # на случай, если попадется тэг, которого нет в маппинге
    except KeyError:
        pass # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    return tagged

Функция для лемматизации токена

In [37]:
def lemmatization(token):
    p = morph.parse(token)[0]
    return p.normal_form

AttributeError: 'Doc' object has no attribute 'lemmatize'

Функция, которая сохраняет токены с информацией о них (в каких постах упоминались и айди синонимов) в формате словаря

In [6]:
def tokens_to_db(texts, posts_id):
    toks = {}
    for a in range(len(texts)):
        doc = Doc(texts[a])
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        morph_vocab = MorphVocab()
        for tok in doc.tokens:
            if tok.pos != 'PUNCT':
                if tok.text.lower() in toks.keys():
                    if posts_id[a] not in toks[tok.text.lower()]['list_posts_id']:
                        toks[tok.text.lower()]['list_posts_id'].append(posts_id[a])
                else:
                    tok.lemmatize(morph_vocab)
                    toks[tok.text.lower()] = {}
                    toks[tok.text.lower()]['list_posts_id'] = [posts_id[a]]
                    toks[tok.text.lower()]['list_synonyms'] = []
                    #for_m = tok.lemma+'_'+tok.pos
                    #try:
                        #for j in model.most_similar(positive=[for_m], topn=2):
                            #toks[tok.text.lower()]['list_synonyms'].append(j[0].split('_')[0])
                    #except:
                        #toks[tok.text.lower()]['list_synonyms'] = []
    tokens_json.update(toks)

Функция, которая сохраняет леммы и информацию о них (к каким токенам принадлежит) в формате словаря

In [26]:
def lemmas_to_db(tokens):
    for tok in tokens:
        lem = lemmatization(tok)
        id_t = tokens_json[tok] 
        if lem in lemmas_json.keys():
            if id_t not in lemmas_json[lem]['list_tokens']:
                lemmas_json[lem]['list_tokens'].append(id_t)
        else:
            lemmas_json[lem] = {}
            lemmas_json[lem]['list_tokens'] = [id_t]

### Подключаемся к бд

In [27]:
con = sqlite3.connect('telegram.db')

In [28]:
c = con.cursor()
con.commit()

Создаём таблицы

In [13]:
c.execute(
        '''CREATE TABLE IF NOT EXISTS tokens (
            id_token integer PRIMARY KEY,
            token text NOT NULL,
            list_posts_id text,
            list_synonyms text);''')
con.commit()

In [8]:
c.execute(
        '''CREATE TABLE IF NOT EXISTS lemmas (
            id_lemma integer PRIMARY KEY,
            lemma text NOT NULL,
            list_tokens text);''')
con.commit()

В несколько потоков обрабатываем токены

In [ ]:
tokens_json = {}
texts = []
posts_id = []
for row in tqdm(c.execute("""SELECT text, posts_id FROM posts""")):
    texts.append(row[-2])
    posts_id.append(row[-1])
threads = []
for i in tqdm(range(0, len(texts), 4000)):
    th_texts = texts[i:i+4000]
    th_posts_id = posts_id[i:i+4000]
    thr = Thread(target=tokens_to_db, args=(th_texts, th_posts_id))
    threads.append(thr)
    thr.start()
for thr in tqdm(threads):
    thr.join()

В несколько потоков обрабатываем леммы

In [31]:
tokens_json = {}
lemmas_json = {}
texts = []
posts_id = []
tokens = []
for row in tqdm(c.execute("""SELECT id_token, token FROM tokens""")):
    if row[-1] not in tokens_json.keys():
        tokens_json[row[-1]] = ''
    tokens_json[row[-1]] = row[-2]
threads = []
for i in tqdm(range(0, len(tokens_json), 10000)):
    th_tokens = [k for k, v in tokens_json.items() if (i<=v)and(v<i+10000)]
    thr = Thread(target=lemmas_to_db, args=(th_tokens,))
    threads.append(thr)
    thr.start()
for thr in tqdm(threads):
    thr.join()

Записываем в бд

In [31]:
tokens_db = tokens_to_db()
i=0
for k, v in tqdm(tokens_json.items()):
    c.execute(
        'INSERT INTO tokens VALUES (?, ?, ?, ?)',
        (i, k, str(v['list_posts_id']), str(v['list_synonyms']))
)
    i+=1
con.commit()

i=0
lemmas_db = lemmas_to_db()
for k, v in tqdm(lemmas_json.items()):
    c.execute(
        'INSERT INTO lemmas VALUES (?, ?, ?)',
        (i, k, str(v['list_tokens']))
    )
    i+=1

con.commit()

KeyboardInterrupt: 

Всё то же самое для токенов (не хватило мощности)

In [73]:
def get_synonyms(tokens):
    for tok in tokens:
        for_m = tag_mystem(tok)
        try:
            for j in model.most_similar(positive=[for_m], topn=2):
                synonyms_db[tok]['list_synonyms'].append(j[0].split('_')[0])
        except:
            synonyms_db[tok] = {}
            synonyms_db[tok]['list_synonyms'] = []

In [ ]:
threads = []
synonyms_db = {}
for i in tqdm(range(0, len(tokens_json), 10000)):
    th_tokens = [k for k, v in tokens_json.items() if (i<=v)and(v<i+10000)]
    thr = Thread(target=get_synonyms, args=(th_tokens,))
    threads.append(thr)
    thr.start()
for thr in tqdm(threads):
    thr.join()

{'analysis': [{'lex': 'сначала', 'wt': 1, 'gr': 'ADV='}], 'text': 'сначала'}
{'analysis': [{'lex': 'энергичность', 'wt': 1, 'gr': 'S,жен,неод=(вин,ед|им,ед)'}], 'text': 'энергичность'}
{'analysis': [{'lex': 'думать', 'wt': 0.9505485925, 'gr': 'V,несов,нп=непрош,вин,ед,прич,полн,жен,действ'}], 'text': 'думающую'}{'analysis': [{'lex': 'долгожданный', 'wt': 1, 'gr': 'A=(дат,мн,полн|твор,ед,полн,муж|твор,ед,полн,сред)'}], 'text': 'долгожданным'}

{'analysis': [{'lex': 'исключение', 'wt': 1, 'gr': 'S,сред,неод=(вин,ед|им,ед)'}], 'text': 'исключение'}
{'analysis': [], 'text': 'pfizer'}
{'analysis': [{'lex': 'выявлять', 'wt': 1, 'gr': 'V,пе=(прош,пр,мн,прич,полн,сов,страд|прош,вин,мн,прич,полн,сов,страд,од|прош,род,мн,прич,полн,сов,страд)'}], 'text': 'выявленных'}
{'analysis': [], 'text': 'yrak'}
{'analysis': [{'lex': 'ход', 'wt': 0.9991030624, 'gr': 'S,муж,неод=(вин,мн|им,мн)'}], 'text': 'ходы'}
{'analysis': [{'lex': 'центр', 'wt': 1, 'gr': 'S,муж,неод=твор,ед'}], 'text': 'центром'}
{'analys

Закрываем бд

In [33]:
#con.close()